## Data Smells

Any time you are given a dataset from anyone, you should immediately be suspicious. Is this data what I think it is? Does it include what I expect? Is there anything I need to know about it? Will it produce the information I expect?

One of the first things you should do is give it the smell test.

Failure to give data the smell test [can lead you to miss stories and get your butt kicked on a competitive story](https://source.opennews.org/en-US/learning/handling-data-about-race-and-ethnicity/).

Let's look at arrest data for Fairfax County, Va. You can find the `arrest.csv` file in this repository.

In [1]:
import agate
arrests = agate.Table.from_csv('arrest.csv')

With data smells, we're trying to find common mistakes in data. For more on data smells, read the GitHub wiki post that started it all. The common mistakes we're looking for are:

* Missing data
* Gaps in data
* Wrong type of data
* Outliers
* Sharp curves
* Conflicting information within a dataset
* Conflicting information across datasets
* Wrongly derived data
* External inconsistency
* Wrong spatial data
* Unusuable data, including non-standard abbreviations, ambigious data, extraneous data, inconsistent data

Not all of these data smells are detectable in code. You may have to ask people about the data. You may have to compare it to another dataset yourself. Does the agency that uses the data produce reports from the data? Does your analysis match those reports? That will expose wrongly derived data, or wrong units, or mistakes you made with inclusion or exclusion.

But with several of these data smells, we can do them first, before we do anything else. First, let's look at **Wrong Type Of Data**. We can sniff that out by simply printing the table structure that Agate has discovered for us.

In [4]:
print arrests

|-----------------+---------------|
|  column_names   | column_types  |
|-----------------+---------------|
|  LName          | Text          |
|  FName          | Text          |
|  MName          | Text          |
|  Age            | Number        |
|  DateArr        | Date          |
|  Charge         | Text          |
|  Charge Descrip | Text          |
|  Address        | Text          |
|-----------------+---------------|



Things seem to look good for this file. The name columns are text, the age column is a number, the date is a date, etc.

The second smell we can find in Agate is Missing Data. We can do that through a series of Group By and Count steps. Let's start with Charge Descrip.

In [5]:
charges = arrests.group_by('Charge Descrip')
median_age = charges.aggregate([('median_age', agate.Median('Age'))])
median_age = age_median.order_by('median_age', reverse=True)

In [6]:
age_median.print_table(max_column_width=50)

|---------------------------------------------------+-------------|
|  Charge Descrip                                   | median_age  |
|---------------------------------------------------+-------------|
|  MOTEL/RESTAURANT/PARK/ETC: FRAUD <$200           |       65.0  |
|  PROTECTIVE ORDER: 3ND VIOL W/IN 20YRS            |       60.0  |
|  PROTECTIVE ORDER: 2ND VIOL W/IN 5 YRS            |       60.0  |
|  DWI: 1ST OFF, BAC >.20%                          |       57.5  |
|  FIREARM: POSSESS BY NON VIOLENT FELON, >10 YRS   |       57.0  |
|  PHONE: ANNOYING RINGING                          |       55.0  |
|  ENTER A SCHOOL AFT CONVICTED OF SEX OFFENSE      |       55.0  |
|  EMERGENCY SERVICES: CALL/SET ALARM W/O CAUSE     |       54.5  |
|  AGGRESSIVE DR: VIOL CERTAIN OFF + HAZARD/HARASS  |       53.0  |
|  MALIC SHOOT OR THROW MISSILE IN/AT OCC. BLDG     |       52.0  |
|  DRUGS: MANUFACTURE, SCH I OR II                  |       51.0  |
|  LARCENY: THIRD OR SUBSEQUENT CONVICTION      

There's a lot of data here, but be sure to focus on the last row - it's blank, and the count is 0, which means there are no rows that are missing a Charge Descrip. That's good, because it means we have no missing data in that column. You can try the same process out using another column like `Age` or `FName`.

Let's now look at **Gaps in Data**. It's been my experience that gaps in data often have to do with time, so let's first look at arrests by month, so we can see what our arrest data covers. You'd expect the number to change, but not by huge amounts. Huge differences could indicate, more often than not, that the data is missing. To do this, we'll need to calculate the month from the date.

In [11]:
arrests_with_months = arrests.compute([
    ('arrest_month', agate.Formula(agate.Text(), lambda row: '%s' % row['DateArr'].month))
])

In [12]:
months = arrests_with_months.group_by('arrest_month')

In [13]:
month_counts = months.aggregate([
    ('count', agate.Count('Charge Descrip'))
])
month_counts = month_counts.order_by('count', reverse=True)

In [14]:
month_counts.print_table()

|---------------+--------|
|  arrest_month | count  |
|---------------+--------|
|  2            | 2,949  |
|  1            |   164  |
|  12           |   130  |
|  11           |    99  |
|  10           |    72  |
|  9            |    60  |
|  8            |    13  |
|  4            |     5  |
|  6            |     4  |
|  5            |     2  |
|  3            |     1  |
|---------------+--------|


This looks a little weird. Yes, the bulk of charges came in February, which is pretty recent, with smaller amounts in January and December. But 60 from September and 72 from October? What's with those? This would mean additional reporting - maybe this is standard practice, or can be easily explained. Maybe an analysis would just include arrest records from the most recent month.

Looks like we should compile some questions to ask the Fairfax County Police Department.

### Assignment

What about the Age column - are there outliers there, and what does Age tell you about the contents of the dataset? Is Address data standardized, or are there variations? Are there errors, like misspellings, in the data? Are there outliers? What are the more interesting and potentially newsworthy charges? What steps in [Agate](https://agate.readthedocs.org/en/1.3.0/) can you take to find out?

Try exploring the data using some of the steps listed above, but with other columns. You can use many of the same functions we have, just change the column name. Add them to this notebook, along with some questions you'd ask the Fairfax police, then add, commit and push this notebook file (`arrests.ipynb`) to Github.

In [5]:
ages = arrests.group_by('Age')
age_counts = ages.aggregate([('age', agate.Count('Age'))])
age_counts = age_counts.order_by('Age', reverse=True)
age_counts.print_table(max_column_width=50)

|------+------|
|  Age | age  |
|------+------|
|   88 |   1  |
|   86 |   1  |
|   83 |   1  |
|   78 |   4  |
|   76 |   3  |
|   73 |   1  |
|   72 |   2  |
|   71 |   3  |
|   70 |   3  |
|   69 |   5  |
|   67 |   5  |
|   66 |   2  |
|   65 |   5  |
|   64 |   4  |
|   63 |   7  |
|   62 |  14  |
|   61 |   5  |
|   60 |  16  |
|   59 |  18  |
|   58 |  14  |
|   57 |  24  |
|   56 |  23  |
|   55 |  29  |
|   54 |  33  |
|   53 |  26  |
|   52 |  28  |
|   51 |  29  |
|   50 |  42  |
|   49 |  27  |
|   48 |  40  |
|   47 |  46  |
|   46 |  31  |
|   45 |  48  |
|   44 |  63  |
|   43 |  43  |
|   42 |  57  |
|   41 |  52  |
|   40 |  36  |
|   39 |  67  |
|   38 |  64  |
|   37 |  72  |
|   36 |  75  |
|   35 |  88  |
|   34 |  71  |
|   33 |  95  |
|   32 | 121  |
|   31 | 124  |
|   30 | 137  |
|   29 | 111  |
|   28 | 119  |
|   27 | 169  |
|   26 | 145  |
|   25 | 160  |
|   24 | 158  |
|   23 | 159  |
|   22 | 177  |
|   21 | 161  |
|   20 | 144  |
|   19 | 167  |
|   18 |

In [8]:
print arrests.rows[0]['Address']


5215 HARBOR COURT DR, ALEXANDRIA, VA 22315


In [10]:
alex = arrests.where(lambda row: 'ALEX' in row['Address'])

In [11]:
print alex.rows[3] ['Address']

3714 NORTH ROSSER STREET, 202, ALEXANDRIA, VA 22311


In [12]:
print len(alex.rows)

656


In [13]:
for row in alex.rows: 
    print row('Address')

TypeError: 'Row' object is not callable

In [10]:
for row in arrests.rows:
    print row['Charge Descrip']

PETIT LARCENY: <$200 NOT FROM A PERSON
LICENSE: DRIVE W/O
LICENSE: DRIVE W/O, 2+ OFFENSE
LICENSE: DRIVE W/O, 2+ OFFENSE
LICENSE: DRIVE W/O
GRAND LARCENY: $200+ NOT FROM A PERSON
BURGLARIOUS TOOLS: POSSESSION OF
IDENTITY THEFT: OBTAIN ID TO DEFRAUD
DRUGS: POSSESS MARIJUANA, 1ST OFF
LIC REVOKED: DR W/O LICENSE, 2 OFF
LIC REVOKED: DR W/O LICENSE, 1 OFF
LICENSE: DRIVE W/O
RECKLE/20 MPH OVER LIMIT
GRAND LARCENY: $200+ NOT FROM A PERSON
GRAND LARCENY: $200+ NOT FROM A PERSON
LIC REVOKED: DR W/O LICENSE, 1 OFF
DRUNK IN PUBLIC OR PROFANE
TRESPASS: AFTER BEING FORBIDDEN TO DO SO
LICENSE: DRIVE W/O, 2+ OFFENSE
FAILURE TO APPEAR: ON FELONY OFFENSE
IDENTITY THEFT: OBTAIN ID TO AVOID ARREST/ETC.
DRUGS: POSSESS SCH I OR II
DRUGS: POSSESS MARIJUANA, 1ST OFF
DRUGS: POSSESS SCH I OR II
DRUGS: DISTRIB/PWI MARIJUANA >1/2 OZ TO 5 LBS
PARAPHERNALIA: UNAUTHORIZED DISTRIBUTION OF
INSUR: OPERATE UNINSURED VEH
INSUR: OPERATE UNINSURED VEH
DRUGS: POSSESS MARIJUANA, 1ST OFF
ASSAULT: ON FAMILY MEMBER
PHONE: INJUR

In [1]:
import agate
arrests = agate.Table.from_csv('arrest.csv')
charges = arrests.group_by('Charge Descrip')
ages_median = charges.aggregate([('ages_median', agate.Median('Age'))])
ages_median.print_table(max_column_width=50)

|-----------------------------------------------------+--------------|
|  Charge Descrip                                     | ages_median  |
|-----------------------------------------------------+--------------|
|  PETIT LARCENY: <$200 NOT FROM A PERSON             |        27.0  |
|  LICENSE: DRIVE W/O                                 |        30.0  |
|  LICENSE: DRIVE W/O, 2+ OFFENSE                     |        28.0  |
|  GRAND LARCENY: $200+ NOT FROM A PERSON             |        29.0  |
|  BURGLARIOUS TOOLS: POSSESSION OF                   |        31.0  |
|  IDENTITY THEFT: OBTAIN ID TO DEFRAUD               |        19.0  |
|  DRUGS: POSSESS MARIJUANA, 1ST OFF                  |        23.0  |
|  LIC REVOKED: DR W/O LICENSE, 2 OFF                 |        30.0  |
|  LIC REVOKED: DR W/O LICENSE, 1 OFF                 |        30.0  |
|  RECKLE/20 MPH OVER LIMIT                           |        27.5  |
|  DRUNK IN PUBLIC OR PROFANE                         |        32.0  |
|  TRE

In [1]:
import agate
arrests = agate.Table.from_csv('arrest.csv')

In [18]:
missing = arrests.where(lambda row:['Charge Descrip'] is None)

In [12]:
print len ('None')

4
